In [33]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/eclipse_bug_records_22-07-20221.csv")
df

<ipython-input-56-470ab6dfcc00>:1: DtypeWarning: Columns (0,3,4,5,19,21,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/eclipse_bug_records_22-07-20221.csv")


,ID,Date 1,Date 2,Count,Occurance,ID2,Application,Package,Type,Version,...,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,1,10-10-2001 21:34,07-06-2021 10:06,1,1,2,Eclipse Project,Platform,Team,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,10-10-2001 21:34,03-04-2018 01:31,1,1,2,Eclipse Project,Platform,Team,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100,10-10-2001 21:36,23-10-2001 23:42,1,1,2,Eclipse Project,Platform,Team,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10000,19-02-2002 11:07,29-09-2012 02:28,1,1,2,Eclipse Project,JDT,UI,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100000,14-06-2005 12:11,05-05-2016 10:31,1,1,1,Eclipse Foundation,z_Archived,TPTP,unspecified,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557938,99995,14-06-2005 12:04,02-09-2005 08:27,1,1,2,Eclipse Project,JDT,UI,3.1,...,not parameterized,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
557939,99996,14-06-2005 12:07,15-06-2005 19:15,1,1,2,Eclipse Project,Platform,Debug,3.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
557940,99997,14-06-2005 12:08,22-09-2006 21:07,1,1,8,WebTools,WTP Source Editing,wst.xsd,1.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
557941,99998,14-06-2005 12:09,03-06-2010 15:10,1,1,1,Eclipse Foundation,z_Archived,TPTP.monitoring,unspecified,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
df.columns

Index(['ID', 'Date 1', 'Date 2', 'Count', 'Occurance', 'ID2', 'Application',
       'Package', 'Type', 'Version', 'Location', 'OS', 'STATUS', 'DECISION',
       'Priority', 'Description', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18',
       'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22',
       'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25'],
      dtype='object')

In [58]:
columns_to_drop = df.columns[16:26]  # Get the column labels from 'Unnamed:16' to 'Unnamed:25'
df.drop(columns=columns_to_drop, inplace=True)

In [59]:
df.columns

Index(['ID', 'Date 1', 'Date 2', 'Count', 'Occurance', 'ID2', 'Application',
       'Package', 'Type', 'Version', 'Location', 'OS', 'STATUS', 'DECISION',
       'Priority', 'Description'],
      dtype='object')

In [60]:
df['Priority'].value_counts()

Priority
normal                                                               375132
enhancement                                                           78601
major                                                                 46800
minor                                                                 19956
critical                                                              16838
blocker                                                                9191
trivial                                                                6604
P3                                                                     3622
P2                                                                      257
P4                                                                      149
 usability                                                              141
P5                                                                      129
P1                                                                      105
 he

In [61]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


df = df[['Description', 'Priority']]
df.dropna(inplace=True)

data = df
priority_counts = data['Priority'].value_counts()

selected_priorities = priority_counts[priority_counts > 19900].index
filtered_df = data[data['Priority'].isin(selected_priorities)]
filtered_df = filtered_df.drop(filtered_df[filtered_df['Priority'] == 'normal'].index)
filtered_df = filtered_df.drop(filtered_df[filtered_df['Priority'] == 'minor'].index)


data = filtered_df
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Convert text to lowercase
    text = str(text)
    text = text.lower()
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords and lemmatize tokens
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    # Join tokens back into a string
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text









[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
<ipython-input-61-65e07aacbb4b>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [62]:


data['Description'] = data['Description'].apply(preprocess_text)

In [63]:
data

,Description,Priority
7,jmx agent,major
18,deleting server runtime allow adding one server,major
46,typing concatenate selected line via key binding,enhancement
50,finish button generic runtime server wizard ne...,major
60,refactoring failed rename class core exception...,major
...,...,...
557910,workbench build rebuild active project,enhancement
557924,dom wrong position boolean type,major
557931,wizard run background option wizard,enhancement
557932,c indentation mode,enhancement


In [64]:
data['Priority'].value_counts()

Priority
enhancement    78601
major          46800
Name: count, dtype: int64

In [65]:
df = data

In [66]:
import pandas as pd
from sklearn.utils import resample


majority_class = 'enhancement'
minority_classes = ['major']

majority_df = df[df['Priority'] == majority_class]
minority_df = df[df['Priority'].isin(minority_classes)]

# Downsample majority class to match the number of samples in minority classes
downsampled_majority_df = resample(majority_df,
                                   replace=False,
                                   n_samples=len(minority_df),
                                   random_state=42)

# Concatenate downsampled majority class with minority classes
downsampled_df = pd.concat([downsampled_majority_df, minority_df])

# Shuffle the dataframe to mix the samples
downsampled_df = downsampled_df.sample(frac=1, random_state=42).reset_index(drop=True)



In [67]:
downsampled_df

,Description,Priority
0,perfmon usability enhancement,enhancement
1,transactioncontroller bitronix,enhancement
2,databinding trigger validation,enhancement
3,eclipse fails start bit linux box,major
4,language support syntax error using ex,enhancement
...,...,...
93595,provide new type jump link named tunnel,enhancement
93596,xmpp chat room url improperly parsed,major
93597,d bind service active,major
93598,adapt egit handle checked exception thrown jgi...,enhancement


In [68]:
data = downsampled_df
X_train, X_test, y_train, y_test = train_test_split(data['Description'], data['Priority'], test_size=0.2, random_state=42)

In [69]:
X_train

25076                  language implement type constructor
27918           slavecontextservice accept null expression
41533                replace usage viewersorter platformui
52525               support xmi serialization xsd instance
2586     exception javalangnosuchmethoderror executing ...
                               ...                        
6265            plan item expose java editor functionality
54886                       repository copy cbi misleading
76820                          fix test warning new launch
860      palette new action change way stereotype displ...
15795    library validation new activexobject supported...
Name: Description, Length: 74880, dtype: object

In [70]:
downsampled_df['Priority'].value_counts()

Priority
enhancement    46800
major          46800
Name: count, dtype: int64

In [71]:
data['Description'] = data['Description'].apply(preprocess_text)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['Description'], data['Priority'], test_size=0.2, random_state=42)

# Vectorize the text data using TF-IDF
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Initialize and train the classifier (e.g., Naive Bayes)
classifier = MultinomialNB()
classifier.fit(X_train_tfidf, y_train)

import joblib
# Save the vectorizer and classifier
joblib.dump(vectorizer, "/content/drive/My Drive/Colab Notebooks/vectorizer.pkl")
joblib.dump(classifier, "/content/drive/My Drive/Colab Notebooks/classifier.pkl")
# Predict bug severity on the testing set
y_pred = classifier.predict(X_test_tfidf)

# Evaluate the model
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

 enhancement       0.80      0.83      0.82      9255
       major       0.83      0.79      0.81      9465

    accuracy                           0.81     18720
   macro avg       0.81      0.81      0.81     18720
weighted avg       0.81      0.81      0.81     18720



In [72]:
data = df

In [51]:
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/eclipse_bug_records_22-07-20221.csv")
df

<ipython-input-51-470ab6dfcc00>:1: DtypeWarning: Columns (0,3,4,5,19,21,22,23,24,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/eclipse_bug_records_22-07-20221.csv")


,ID,Date 1,Date 2,Count,Occurance,ID2,Application,Package,Type,Version,...,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,1,10-10-2001 21:34,07-06-2021 10:06,1,1,2,Eclipse Project,Platform,Team,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,10-10-2001 21:34,03-04-2018 01:31,1,1,2,Eclipse Project,Platform,Team,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100,10-10-2001 21:36,23-10-2001 23:42,1,1,2,Eclipse Project,Platform,Team,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10000,19-02-2002 11:07,29-09-2012 02:28,1,1,2,Eclipse Project,JDT,UI,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100000,14-06-2005 12:11,05-05-2016 10:31,1,1,1,Eclipse Foundation,z_Archived,TPTP,unspecified,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557938,99995,14-06-2005 12:04,02-09-2005 08:27,1,1,2,Eclipse Project,JDT,UI,3.1,...,not parameterized,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
557939,99996,14-06-2005 12:07,15-06-2005 19:15,1,1,2,Eclipse Project,Platform,Debug,3.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
557940,99997,14-06-2005 12:08,22-09-2006 21:07,1,1,8,WebTools,WTP Source Editing,wst.xsd,1.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
557941,99998,14-06-2005 12:09,03-06-2010 15:10,1,1,1,Eclipse Foundation,z_Archived,TPTP.monitoring,unspecified,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

df = df[['Description', 'Priority']]
df.dropna(inplace=True)

data = df
priority_counts = data['Priority'].value_counts()

selected_priorities = priority_counts[priority_counts > 19900].index
filtered_df = data[data['Priority'].isin(selected_priorities)]
filtered_df = filtered_df.drop(filtered_df[filtered_df['Priority'] == 'normal'].index)
filtered_df = filtered_df.drop(filtered_df[filtered_df['Priority'] == 'minor'].index)

data = filtered_df
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # Convert text to lowercase
    text = str(text)
    text = text.lower()
    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords and lemmatize tokens
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    # Join tokens back into a string
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text









[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
<ipython-input-52-65e07aacbb4b>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [53]:
data['Description'] = data['Description'].apply(preprocess_text)

In [54]:
data['Priority'].value_counts()

Priority
enhancement    78601
major          46800
Name: count, dtype: int64

In [55]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report

X = data['Description']
y = data['Priority']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=40)

# Convert text data into numerical representation using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)


smote = SMOTE()

# Upsample the minority classes using SMOTE
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_tfidf, y_train)

# Train a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_resampled, y_train_resampled)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test_tfidf)

# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Confusion Matrix:
[[19470  3967]
 [ 3949 10235]]

Classification Report:
              precision    recall  f1-score   support

 enhancement       0.83      0.83      0.83     23437
       major       0.72      0.72      0.72     14184

    accuracy                           0.79     37621
   macro avg       0.78      0.78      0.78     37621
weighted avg       0.79      0.79      0.79     37621

